In [ ]:
import instructor
from openai import OpenAI, AsyncOpenAI
from langsmith.wrappers import wrap_openai

client = OpenAI()

# Custom wrap for VSCode, needs to be the first wrap!
from patch_openai.patch_openai import patch_openai
client = patch_openai(client)

# Wrap the OpenAI client with LangSmith
client = wrap_openai(client)

client = instructor.from_openai(client, mode=instructor.Mode.TOOLS)

In [ ]:
import instructor
from pydantic import BaseModel, field_validator

class UserDetail(BaseModel):
    name: str
    age: int

    @field_validator("name")
    @classmethod
    def name_must_be_uppercase(cls, name):
        if name.islower():
            raise ValueError("name must be UPPERCASE. Use the tool again but modify the name argument to 'JASON'.")
        return name

response = client.chat.completions.create(
    model="gpt-4o-mini",
    response_model=UserDetail,
    temperature=0.01,
    messages=[
        {"role": "user", "content": "Extract `jason is 12`"},
    ],
    max_retries=2,
)

print(response.model_dump_json(indent=2))